<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/makePlan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install peft
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.8 MB/s eta 0:00:00


In [ ]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset

In [ ]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model_path = "meta-llama/Llama-3.1-8B-Instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    # device_map="auto",
    # trust_remote_code=True,
    # use_auth_token=True,
)

model.eval()

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [ ]:
dataset = load_dataset("koutch/staqc", 'sca_python')
dataset

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'snippet'],
        num_rows: 85294
    })
})

In [ ]:
model.eval()

from tqdm import tqdm
import csv

csv_file = "generated_similarQuestion.csv"

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

start_row = 0
max_snippet_length = 1024
sliced_dataset = dataset['train'].select(range(0, 100))

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["question_id", "question", "snippet", "similarQuestion"])

    for row in tqdm(sliced_dataset, desc="Generating plans", unit="sample"):
        question_id = row['question_id']
        question = row['question']
        snippet = row['snippet']

        if len(snippet) > max_snippet_length:
            snippet = snippet[:max_snippet_length]

        prompt = f"""Given a competitive programming problem generate a concrete planning to solve the problem.
The problem is provided along with a Python code snippet that can be used as a reference to help solve the problem.
Please generalize the provided code so that it can be applied to similar problems, and summarize the plan in 6 lines or less.
Focusing only on the key actions needed to construct the code in text form, not on generating the code itself.
Importnat: you should give only the planning to solve the problem. Do not add extra explanation or words.
Once the plan is complete, indicate the end by generating <|eot_id|> token.

### Original Question :
{question}
### Code Snippet :
{snippet}
### Planning to solve the problem :
"""

        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=2048,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.2
            )

            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            keyword = "### Planning to solve the problem :"
            if keyword in generated_text:
                generated_question = generated_text.split(keyword)[1].strip()

            keyword = "###"
            if keyword in generated_question:
                generated_question = generated_question.split(keyword)[0].strip()

            writer.writerow([question_id, question, snippet, generated_question])



Generating similar questions: 100%|██████████| 100/100 [28:55<00:00, 17.35s/sample]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp generated_similarQuestion.csv /content/drive/MyDrive/


In [ ]:
import pandas as pd

df = pd.read_csv('generated_similarQuestion.csv')
print(df['question'][0])
print(df['similarQuestion'][0])

Using Regex (or simmilar) to change song name (or other tag) in itunes via script
How to modify song names in iTunes by replacing a specific substring with another using a script that utilizes AppleScript's text item delimiters?
